# Numerical Benchmarks

In [49]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [50]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
d = 2**n
rank = 64
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = DilutedKrausMap(
                               U=generate_unitary(d=d), 
                               c=0.1, 
                               d=d, 
                               rank=rank-1,
                               spam = spam_target
                              )

### Generate Synthetic Data with Noise

In [51]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = 5000, 
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_channel(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.0316)

### Fit Model, Initial and Corruption Matrix

In [52]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = 500,
                 verbose = False,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

1.0550758972756566e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.001072907753846531


In [53]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 64,
                                           spam = spam_model,
                                          ),
                        loss_function = KLDiv(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )

  0%|          | 0/10000 [00:00<?, ?it/s]

None -0.16780988097723637
None -0.26674813996925634
None -0.3419568748355236
None -0.3891446310210777
None -0.42231469560478435
None -0.4465347562160434
None -0.4656463975215702
None -0.48087926185882607
None -0.49294690252808476
None -0.5031881734335782
None -0.5104772941325172
None -0.5172591882216468
None -0.5229599085982989
None -0.5281885550458559
None -0.5317527301452911
None -0.5349778581086432
None -0.5377034869142214
None -0.5404497657578758
None -0.5431559428345646
None -0.5449811471064804
None -0.5465998740385747
None -0.5477489281271896
None -0.5489306331350335
None -0.5501321192049697
None -0.5515586760950576
None -0.5520380088137135
None -0.5528410625312462
None -0.5535615260124928
None -0.5540213407555338
None -0.5544634119515378
None -0.554749161137032
None -0.5553445616537434
None -0.5556114606286129
None -0.555787740608114
None -0.5562191032645227
None -0.5561998718669611
None -0.5566050820417388
None -0.5570919614136308
None -0.5576605652790795
None -0.55749165034957

In [54]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 64,
                                           spam = spam_model,
                                          ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )

  0%|          | 0/10000 [00:00<?, ?it/s]

None -0.16780988097723637
None -0.2553283851180475
None -0.329430671795595
None -0.3771333169656544
None -0.4112015905049336
None -0.436570603836224
None -0.45677750567472414
None -0.47292088217026584
None -0.48577574593700545
None -0.4967855716669943
None -0.5048587387204345
None -0.5122517359234304
None -0.5183896322833258
None -0.5240007313230264
None -0.5279896484724009
None -0.5316057631791803
None -0.5345441884209929
None -0.5374848142821576
None -0.5404643679007776
None -0.5424814430298858
None -0.5442583936917778
None -0.5456541142595757
None -0.5470169331065357
None -0.5483986924798854
None -0.5500003312182254
None -0.5505035098007105
None -0.5514581753738679
None -0.5523092543514374
None -0.5528788486847586
None -0.5533891436708867
None -0.5537683667836397
None -0.5545023383715286
None -0.554831818972755
None -0.5550618662308734
None -0.5554873541932159
None -0.5555177179269811
None -0.5559966906651929
None -0.5564726343890519
None -0.5571025830945289
None -0.5568998961120265

In [55]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = 10000, 
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_channel(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.0316)

In [56]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = 500,
                 verbose = False,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

1.0550758972756566e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.001072907753846531


In [57]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 64,
                                           spam = spam_model,
                                          ),
                        loss_function = KLDiv(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )

  0%|          | 0/10000 [00:00<?, ?it/s]

None -0.16780988097723637
None -0.27039923463633747
None -0.35269281541534364
None -0.407642568180476
None -0.4487545530547771
None -0.48099700546232577
None -0.5067491578719732
None -0.5279712405788568
None -0.5458721047537027
None -0.5604879544957403
None -0.573288057371169
None -0.5845326650667382
None -0.5936705240911263
None -0.601453264850099
None -0.6084833785038131
None -0.6147585076303279
None -0.6204753060949331
None -0.6251401631342643
None -0.6296349374479344
None -0.6334837233532437
None -0.6369772559732984
None -0.6397986493550535
None -0.6428494338421821
None -0.645702632881311
None -0.6477027738806314
None -0.6500460674487238
None -0.6521930965984143
None -0.6535659814440737
None -0.6556058957138592
None -0.6568070251564528
None -0.6584638584170838
None -0.6595918815819853
None -0.6604280026666638
None -0.6615836195350506
None -0.6620598629671165
None -0.6627420245723125
None -0.6632435956487062
None -0.6636948955432017
None -0.6644204375909365
None -0.6647212489239074


In [58]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 64,
                                           spam = spam_model,
                                          ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )

  0%|          | 0/10000 [00:00<?, ?it/s]

None -0.16780988097723637
None -0.25843458647263723
None -0.33892901415275356
None -0.3937145078575534
None -0.4352359281832246
None -0.4679997194938583
None -0.49446343767643736
None -0.5165255287088448
None -0.5351291887436259
None -0.5504937246272056
None -0.5640240930068524
None -0.5759104458563893
None -0.5856747981966895
None -0.5939736704545457
None -0.6014291737685574
None -0.6080447414583074
None -0.6140912698453411
None -0.6191562083458018
None -0.623912897380084
None -0.6280025198177811
None -0.6317952491756941
None -0.6347804934501351
None -0.6381283187285561
None -0.6410818788911075
None -0.6432477105772324
None -0.6457856366217992
None -0.6480534125444288
None -0.6496000120954982
None -0.6516822108862531
None -0.6530787887438115
None -0.6548246614569946
None -0.6560592848102728
None -0.6570518444643848
None -0.6582376711969934
None -0.6588131697889583
None -0.6597440722479307
None -0.6603031361032736
None -0.6607891451594915
None -0.6615409489052366
None -0.66199974973409